In [2]:
!pip install transformers datasets torch peft accelerate bitsandbytes

  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached torch-2.7.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached accelerate-1.8.1-py3-none-any.whl.metadata (19 kB)
  Using cached bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached regex-2024.11.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached tokenizers-0.21.2-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached safetensors-0.5.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using ca

In [3]:
!pip uninstall keras -y  # Remove Keras 3 if installed
!pip install tf-keras    # Install TensorFlow-compatible Keras

Found existing installation: keras 3.9.2
Uninstalling keras-3.9.2:
  Successfully uninstalled keras-3.9.2
  Using cached tf_keras-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorflow-2.19.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.10.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached ml_dtypes-0.5.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (21 kB)
Using cached tf_keras-2.19.0-py3-none-any.whl (1.7 MB)
Using cached tensorflow-2.19.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (645.0 MB)
Using cached keras-3.10.0-py3-none-any.whl (1.4 MB)
Using cached ml_dtypes-0.5.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.7 MB)
Using cached tensorboard-2.19.0-py3-none-any.whl (5.5 MB)
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:

In [4]:
!pip install trl

  Using cached trl-0.19.0-py3-none-any.whl.metadata (10 kB)
Using cached trl-0.19.0-py3-none-any.whl (375 kB)


In [5]:
!pip install triton

In [6]:
!pip uninstall bitsandbytes -y
!pip install bitsandbytes

Found existing installation: bitsandbytes 0.46.1
Uninstalling bitsandbytes-0.46.1:
  Successfully uninstalled bitsandbytes-0.46.1
  Using cached bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl (72.9 MB)


In [7]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
from datasets import load_dataset

# Load dataset
dataset = load_dataset("json", data_files="../Datos/chatbot_data.json", split="train")

# Model & Tokenizer
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# Quantization (4-bit + CPU offload)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    llm_int8_enable_fp32_cpu_offload=True
)

# Load model with offloading
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model = prepare_model_for_kbit_training(model)

# LoRA Config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(model, peft_config)

# Training Arguments (without max_seq_length)
training_args = TrainingArguments(
    output_dir="./deepseek-chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=50,
    logging_steps=10,
    save_steps=100,
    fp16=True,
    optim="paged_adamw_8bit",
    report_to="none",
)

# Formatting function
def formatting_func(example):
    # Adjust this based on your dataset structure
    # For simple text:
    return {"text": example["messages"]}
    
    # If your data has chat format (user/assistant messages):
    # text = ""
    # for message in example["messages"]:
    #     text += f"{message['role']}: {message['content']}\n"
    # return {"text": text.strip()}

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_args,
    formatting_func=formatting_func,  # Replaces dataset_text_field
)

# Train!
trainer.train()
trainer.save_model("deepseek-chatbot-finetuned")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,3.203900
20,1.974900
30,1.327600
40,1.131000
50,1.081900


In [8]:
from transformers import pipeline

# Load fine-tuned model
model_path = "deepseek-chatbot-finetuned"
chatbot = pipeline(
    "text-generation",
    model=model_path,
    tokenizer=tokenizer,
    device="cuda",
)

# Chat example
prompt = """<|system|>You are a helpful assistant.</s>
<|user|>How do I make coffee?</s>
<|assistant|>"""

output = chatbot(
    prompt,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
)
print(output[0]['generated_text'])

Device set to use cuda


<|system|>You are a helpful assistant.</s>
<|user|>How do I make coffee?</s>
<|assistant|>Make coffee? That's a bit tricky. You need to have a good coffee machine, but I don't have that. You could make your own at home, but I don't know if it'll taste right. Maybe you could try using some local ingredients and some coffee beans. Or you could look up a recipe online. I don't have a recipe for a home made coffee maker. You could also visit a coffee shop and ask them how they make their own coffee. Or maybe you could
